## Loading libraries

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import random as rn

In [ ]:
#Reproducible Results
import os
os.environ['PYTHONHASHSEED'] = str(1337)

In [ ]:
#Reproducible Results
np.random.seed(1337)

In [ ]:
#Reproducible Results
rn.seed(1337)

In [ ]:
#Reproducible Results
tf.compat.v1.set_random_seed(1337)

In [ ]:
#Reproducible Results
from keras import backend as K

session_conf = tf.compat.v1.ConfigProto(
    intra_op_parallelism_threads=1, 
    inter_op_parallelism_threads=1
)

sess = tf.compat.v1.Session(
    graph=tf.compat.v1.get_default_graph(), 
    config=session_conf
)

tf.compat.v1.keras.backend.set_session(sess)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# To display the whole columns
pd.set_option('display.max_columns', 999)

low_memory=False

In [ ]:
# Plotting
import seaborn as sns
import matplotlib.pyplot as plt

## Reading Data
### Fahad

In [ ]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Dataset is now stored in a Pandas dataframe

#Fahad,I will add the data to the dataframe
train_features = pd.read_csv('/kaggle/input/lish-moa/train_features.csv')
train_targets =  pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')
test_features = pd.read_csv('/kaggle/input/lish-moa/test_features.csv')

In [ ]:
# To use them later in sumbission dataframe
columns_names = list(train_targets.columns)
sig_id = test_features[['sig_id']]

## EDA 
### Haifaa

In [ ]:
# Most of observations have only one label or no label, 
# and the others have more than 1 label  

sns.set(style="whitegrid")
sns.countplot(x=train_targets[train_targets == 1].sum(axis=1), color ="b")
plt.title("The Number of Lables Compared to Observations")
plt.xlabel("Number of Labels")
plt.ylabel("Number of Observations")

In [ ]:
# Most of the labels are triggered by only one observation, 
# some of them have more than 4 observations
sns.set(style="whitegrid")
plt.figure(figsize=(30,8))
sns.countplot(x=train_targets[train_targets == 1].sum(axis=0), color ='r')
plt.title("The Distribution of Observations Among Lables")
plt.xlabel("The Label")
plt.ylabel("Number of Observations")
plt.xticks(rotation=90)

## Pre-processing
### Haifaa

In [ ]:
from sklearn.preprocessing import LabelEncoder, LabelBinarizer

In [ ]:
# Remove sig_id
train_features = train_features.drop(['sig_id'], axis=1)
train_targets = train_targets.drop(['sig_id'], axis=1)

test_features = test_features.drop(['sig_id'], axis=1)

In [ ]:
# Label encoder 
lb_enc = LabelEncoder()

train_features['cp_type'] = lb_enc.fit_transform(train_features['cp_type'])
train_features['cp_dose'] = lb_enc.fit_transform(train_features['cp_dose'])

test_features['cp_type'] = lb_enc.fit_transform(test_features['cp_type'])
test_features['cp_dose'] = lb_enc.fit_transform(test_features['cp_dose'])

In [ ]:
# Convert into arrays
train_features = train_features.values
train_targets = train_targets.values

test_features = test_features.values

In [ ]:
# Reshape the features 
x_train_flat = train_features.reshape(-1,875)
x_test_flat = test_features.reshape(-1,875)

In [ ]:
train_features.shape

## PCA
### Haifaa

In [ ]:
# Using PCA to keep most 90% important features
from sklearn.decomposition import PCA
pca = PCA(0.9)

pca.fit(x_train_flat)

PCA(copy=True, iterated_power='auto', n_components=0.9, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

pca.n_components_

train_pca = pca.transform(x_train_flat)
test_pca = pca.transform(x_test_flat)

In [ ]:
test_pca.shape

## Network Design (DNN)
### Haifaa

In [ ]:
# Loading required libraries for DL
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [ ]:
# Hyper-parameters
batch_size = 100
num_classes = 206
epochs = 200

In [ ]:
'''
from sklearn.model_selection import KFold

# define 10-fold cross validation test harness
kfold = KFold(n_splits=10, shuffle=True, random_state=1337)

cvscores = []

for train, test in kfold.split(train_features, train_targets):
    
    x_train, x_test = train_features.iloc[list(train)], train_features.iloc[list(test)]
    y_train, y_test = train_targets.iloc[list(train)], train_targets.iloc[list(test)]

    # The model architecture
    model = Sequential()
    model.add(Dense(1024, activation='tanh', input_shape=(875,)))
    model.add(Dense(512, activation='tanh'))
    model.add(Dense(512, activation='tanh'))
    model.add(Dense(256, activation='tanh'))
    model.add(Dense(num_classes, activation='sigmoid'))
    
    # Compile model
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy', f1_m])
    
    # Train the model
    history = model.fit(x_train, 
                        y_train,
                        batch_size=batch_size,
                        epochs=10,
                        callbacks=callbacks,
                        verbose=1)                         
    
    # Evaluate the model
    scores = model.evaluate(x_test, y_test, verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
'''

In [ ]:
# The network
model = Sequential()
model.add(Dense(1024, activation='tanh', input_shape=(280,)))
model.add(Dense(512, activation='tanh'))
model.add(Dense(512, activation='tanh'))
model.add(Dense(256, activation='tanh'))
model.add(Dense(num_classes, activation='sigmoid'))

In [ ]:
# To monitor validation set results , we will use F1 score scince we have multi-label classification
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy', f1_m])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

callbacks = [EarlyStopping(monitor='loss', mode='min', patience=5, verbose=1),
             ModelCheckpoint('MoA.hdf5', monitor='loss', mode='min', 
                             verbose=1, save_best_only=True, save_weights_only=False, period=1)]

In [ ]:
# Training
history = model.fit(train_pca, 
                    train_targets,
                    batch_size=batch_size,
                    epochs=epochs,
                    callbacks=callbacks,
                    verbose=1)

## Prediction
### Haifaa

In [ ]:
# Obtain the predictions for test set
predictions = model.predict(test_pca)

In [ ]:
predictions

In [ ]:
# Convert the predictions into dataframe
submission = pd.DataFrame(data=predictions)

In [ ]:
# Add sig_id to the results
submission.insert(loc=0, column='1', value=sig_id)

In [ ]:
# Getting column names for the submission dataframe
submission.columns = columns_names

In [ ]:
submission.head()

In [ ]:
#submission.iloc[:,1:] = submission.iloc[:,1:].round(2)

In [ ]:
#submission.head()

In [ ]:
# Save submission file into a csv
submission.to_csv('/kaggle/working/submission.csv', index = False)

  ## Team Members:
  
  1- EDA (Fahad)  >> Not implemented yet
  
  2- Statistical Summary + Data Visualization (Bashayer) >> Not implemented yet
  
  3- Feature Engineering (Faris) >> Withdrown 
  
  4- Pre-processing (Kholoud) >> Withdrown 

  5- Pre-processing + Building the model (Haifaa) >> Done
  
  6- Deploying the model = Sample Submission